# LangChain的Memory的使用

在web页面和llm交互的时候，llm能够记住之前的对话，是因为每次对话，会发送之前所有的聊天记录给LLM。
LangChain已经做了很好的封装，代码在`langchain-guide/lib/python3.11/site-packages/langchain/memory/__init__.py`

这里介绍一些常见的库,下面全部用`ConversationChain`来演示

对话的实现的原理如下：
- 在发送给llm之前和之后保存message
- 调整promot，在发送的时候获取聊天记录，格式化到promot中，发送给llm
- 需要做隔离，通过session_id 来获取对应的客户的聊天记录。

## ConversationBufferMemory

将对话记录存储在内存中，并且不会删减任何的对话记录

In [3]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain_openai import OpenAI

memory = ConversationBufferMemory()
conversation = ConversationChain(llm=OpenAI(),memory=memory,verbose=True)
conversation.invoke({"input":"你好，我叫ethan"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 你好，我叫ethan
AI:

> Finished chain.


{'input': '你好，我叫ethan',
 'history': '',
 'response': ' 你好，ethan！很高兴认识你。我是一名AI助手，可以回答你关于科学和技术的问题。你有什么想要问我的吗？'}

In [4]:
conversation.invoke({"input":"我叫什么"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 你好，我叫ethan
AI:  你好，ethan！很高兴认识你。我是一名AI助手，可以回答你关于科学和技术的问题。你有什么想要问我的吗？
Human: 我叫什么
AI:

> Finished chain.


{'input': '我叫什么',
 'history': 'Human: 你好，我叫ethan\nAI:  你好，ethan！很高兴认识你。我是一名AI助手，可以回答你关于科学和技术的问题。你有什么想要问我的吗？',
 'response': ' 你的名字是ethan，我刚刚在你打招呼的时候就知道了。我可以帮你记住一些重要的事情，比如你的名字和喜欢的事物。你可以随时告诉我。'}

In [6]:
# 存储的对话的记忆
memory.buffer_as_str

'Human: 你好，我叫ethan\nAI:  你好，ethan！很高兴认识你。我是一名AI助手，可以回答你关于科学和技术的问题。你有什么想要问我的吗？\nHuman: 我叫什么\nAI:  你的名字是ethan，我刚刚在你打招呼的时候就知道了。我可以帮你记住一些重要的事情，比如你的名字和喜欢的事物。你可以随时告诉我。'

## ConversationBufferWindowMemory

限制了message的数量。比如2，他会记录两次对话记录，一共4条记录。

In [27]:
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationChain
from langchain_openai import OpenAI

memory = ConversationBufferWindowMemory(k=2)
conversation = ConversationChain(llm=OpenAI(),memory=memory,verbose=True)
conversation.invoke({"input":"你好，我叫ethan"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 你好，我叫ethan
AI:

> Finished chain.


{'input': '你好，我叫ethan',
 'history': '',
 'response': ' 你好，ethan！很高兴认识你。我是一个人工智能助手，专门为用户提供帮助和解答问题的。你有什么需要我的帮助吗？'}

In [28]:
len(memory.buffer_as_messages)

2

In [29]:
conversation.invoke({"input":"我叫什么"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 你好，我叫ethan
AI:  你好，ethan！很高兴认识你。我是一个人工智能助手，专门为用户提供帮助和解答问题的。你有什么需要我的帮助吗？
Human: 我叫什么
AI:

> Finished chain.


{'input': '我叫什么',
 'history': 'Human: 你好，我叫ethan\nAI:  你好，ethan！很高兴认识你。我是一个人工智能助手，专门为用户提供帮助和解答问题的。你有什么需要我的帮助吗？',
 'response': ' 你的名字是ethan，是吗？如果不是，请告诉我你的真实名字，我会记住它的。'}

In [30]:
len(memory.buffer_as_messages)

4

In [31]:
conversation.invoke({"input":"我第一句话说的是什么"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 你好，我叫ethan
AI:  你好，ethan！很高兴认识你。我是一个人工智能助手，专门为用户提供帮助和解答问题的。你有什么需要我的帮助吗？
Human: 我叫什么
AI:  你的名字是ethan，是吗？如果不是，请告诉我你的真实名字，我会记住它的。
Human: 我第一句话说的是什么
AI:

> Finished chain.


{'input': '我第一句话说的是什么',
 'history': 'Human: 你好，我叫ethan\nAI:  你好，ethan！很高兴认识你。我是一个人工智能助手，专门为用户提供帮助和解答问题的。你有什么需要我的帮助吗？\nHuman: 我叫什么\nAI:  你的名字是ethan，是吗？如果不是，请告诉我你的真实名字，我会记住它的。',
 'response': ' 你第一句话说的是“你好，我叫ethan”。'}

In [32]:
len(memory.buffer_as_messages)

4

In [33]:
conversation.invoke({"input":"我第一句话说的是什么"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 我叫什么
AI:  你的名字是ethan，是吗？如果不是，请告诉我你的真实名字，我会记住它的。
Human: 我第一句话说的是什么
AI:  你第一句话说的是“你好，我叫ethan”。
Human: 我第一句话说的是什么
AI:

> Finished chain.


{'input': '我第一句话说的是什么',
 'history': 'Human: 我叫什么\nAI:  你的名字是ethan，是吗？如果不是，请告诉我你的真实名字，我会记住它的。\nHuman: 我第一句话说的是什么\nAI:  你第一句话说的是“你好，我叫ethan”。',
 'response': '  我已经告诉你了，你第一句话说的是“你好，我叫ethan”。'}

## ConversationSummaryBufferMemory

他支持token数量，超过指定token后,会调用llm生成summary。

In [39]:
from langchain.memory import ConversationSummaryBufferMemory
# 方便演示，直接调用保存，不通过llm跑整个流程了
memory = ConversationSummaryBufferMemory(
    llm = OpenAI(),
    max_token_limit=10,
    return_messages=True
)
memory.save_context(
    {"input":"帮我找一下tomie"},
    {"output":"对不起请问什么是tomie？"}
)
memory.save_context(
    {"input":"tomie是一个培训讲师"},
    {"output":"好的，我知道了。"}
)
memory.save_context(
    {"input":"今天他要讲一门关于RAG的课程"},
    {"output":"好的，我知道了。需要RAG的资料吗？"}
)

In [40]:
memory.load_memory_variables({})

{'history': [SystemMessage(content='\nThe human requests for the AI to find information on "tomie", but the AI is not familiar with this term and asks for clarification. The human explains that "tomie" is a training instructor. The AI acknowledges and says it understands. The human then requests for the AI to find information on a course about RAG that will be taught today. The AI agrees and asks if any materials on RAG are needed.')]}

## ConversationTokenBufferMemory

指定token的长度，他会保留最新的满足token 的对话

In [41]:
from langchain.memory import ConversationTokenBufferMemory

memory = ConversationTokenBufferMemory(llm=OpenAI(),max_token_limit=150)
memory.save_context(
    {"input":"帮我找一下tomie"},
    {"output":"对不起请问什么是tomie？"}
)
memory.save_context(
    {"input":"tomie是一个培训讲师"},
    {"output":"好的，我知道了。"}
)
memory.save_context(
    {"input":"今天他要讲一门关于RAG的课程"},
    {"output":"好的，我知道了。需要RAG的资料吗？"}
)
memory.save_context(
    {"input":"不需要资料了，谢谢"},
    {"output":"好的，那我就不打扰你了。"}
)

In [44]:
memory.buffer_as_messages

[HumanMessage(content='帮我找一下tomie'),
 AIMessage(content='对不起请问什么是tomie？'),
 HumanMessage(content='tomie是一个培训讲师'),
 AIMessage(content='好的，我知道了。'),
 HumanMessage(content='今天他要讲一门关于RAG的课程'),
 AIMessage(content='好的，我知道了。需要RAG的资料吗？'),
 HumanMessage(content='不需要资料了，谢谢'),
 AIMessage(content='好的，那我就不打扰你了。')]

In [45]:
memory.save_context(
    {"input":"不需要资料了，谢谢"},
    {"output":"好的，那我就不打扰你了1。"}
)
memory.save_context(
    {"input":"不需要资料了，谢谢"},
    {"output":"好的，那我就不打扰你了2。"}
)
# 可以看到，之前的消息已经么了
memory.buffer_as_messages

[AIMessage(content='好的，我知道了。'),
 HumanMessage(content='今天他要讲一门关于RAG的课程'),
 AIMessage(content='好的，我知道了。需要RAG的资料吗？'),
 HumanMessage(content='不需要资料了，谢谢'),
 AIMessage(content='好的，那我就不打扰你了。'),
 HumanMessage(content='不需要资料了，谢谢'),
 AIMessage(content='好的，那我就不打扰你了1。'),
 HumanMessage(content='不需要资料了，谢谢'),
 AIMessage(content='好的，那我就不打扰你了2。')]

## 使用知识图谱构建记忆

`ConversationKGMemory`会使用外部的知识图谱，它会将对话传递给llm，让llm提取知识图谱的三元组，构建知识图谱。
知识三元组是包含主语、谓语和宾语的子句。 主语是被描述的实体，谓语是正在描述的主体属性，而宾语则是该属性的值。

In [68]:
from langchain.llms import  OpenAI
from langchain.memory import ConversationKGMemory

llm = OpenAI(
    temperature=0
)   

memory = ConversationKGMemory(llm=llm,return_messages=True)

memory.save_context(
    {"input":"帮我找一下tomie"},
    {"output":"对不起请问什么是tomie？"}
)

memory.save_context(
    {"input":"tomie是一个培训讲师"},
    {"output":"好的，我知道了。"}
)

In [69]:
memory.load_memory_variables({"input":"tomie是谁?"})
# 这个返回值说明 这句话是在 tomie的实体上，message是 tomie is a training lecturer.')

{'history': [SystemMessage(content='On tomie: tomie is a training lecturer.')]}

In [70]:
# 这句话和tomie有关，返回的是tomie实体
memory.get_current_entities("tomie最喜欢做什么事?")

['tomie']

In [72]:
# 实体是Ethan
memory.get_current_entities("ethan是谁?")

['Ethan']

In [73]:
# 生成了三元组，
memory.get_knowledge_triplets("tomie最喜欢打游戏")

[KnowledgeTriple(subject='tomie', predicate='最喜欢', object_='打游戏')]

In [74]:
# 这句话没有里面，没有主谓宾，直接返回空
memory.get_knowledge_triplets("1+1等于什么")

[]

In [63]:
from langchain.memory.prompt import KNOWLEDGE_TRIPLE_EXTRACTION_PROMPT
# 构建知识图片的prompt
print(KNOWLEDGE_TRIPLE_EXTRACTION_PROMPT.invoke({"history":"","input":"input"}).to_string())

You are a networked intelligence helping a human track knowledge triples about all relevant people, things, concepts, etc. and integrating them with your knowledge stored within your weights as well as that stored in a knowledge graph. Extract all of the knowledge triples from the last line of conversation. A knowledge triple is a clause that contains a subject, a predicate, and an object. The subject is the entity being described, the predicate is the property of the subject that is being described, and the object is the value of the property.

EXAMPLE
Conversation history:
Person #1: Did you hear aliens landed in Area 51?
AI: No, I didn't hear that. What do you know about Area 51?
Person #1: It's a secret military base in Nevada.
AI: What do you know about Nevada?
Last line of conversation:
Person #1: It's a state in the US. It's also the number 1 producer of gold in the US.

Output: (Nevada, is a, state)<|>(Nevada, is in, US)<|>(Nevada, is the number 1 producer of, gold)
END OF EXAM

In [67]:
memory.clear()

## 构建记忆实体概念清单

他会将对话中的实体区分出来，对每个实体做总结。他的对话记录默认是存储在内存中的，可以替代。
在使用的时候需要先调用`load_memory_variables`方法来解析对话，生成实体和总结

In [97]:
from langchain.llms import  OpenAI
from langchain.memory import ConversationEntityMemory

llm = OpenAI(
    temperature=0
)

memory = ConversationEntityMemory(llm=llm)
_input = {
    "input":"飞流和小王子，张狗屁是好朋友，他有有个组合叫做创造303"
}
# 需要先生成实体和总结
memory.load_memory_variables(_input)
memory.save_context(
    _input,
    {
        "output":"哇，我想也和你们做朋友"
    }
)


In [98]:
memory.entity_cache

['飞流', '小王子', '张狗屁', '创造303']

In [99]:
memory.entity_store

InMemoryEntityStore(store={'飞流': '飞流和小王子，张狗屁是好朋友，他有有个组合叫做创造303', '小王子': '小王子和张狗屁是好朋友，他们一起组成了创造303组合。', '张狗屁': '张狗屁和飞流、小王子是好朋友，他们一起有个组合叫做创造303。', '创造303': '创造303是由张狗屁组成的组合。'})

In [101]:
memory.load_memory_variables({"input":"创造303是什么?"})

{'history': 'Human: 飞流和小王子，张狗屁是好朋友，他有有个组合叫做创造303\nAI: 哇，我想也和你们做朋友',
 'entities': {'创造303': '创造303是由张狗屁组成的组合。'}}

In [104]:
from langchain.memory.prompt import ENTITY_EXTRACTION_PROMPT

# 生成实体的prompt
print(ENTITY_EXTRACTION_PROMPT.invoke({"history": [], "input": ""}).to_string())

You are an AI assistant reading the transcript of a conversation between an AI and a human. Extract all of the proper nouns from the last line of conversation. As a guideline, a proper noun is generally capitalized. You should definitely extract all names and places.

The conversation history is provided just in case of a coreference (e.g. "What do you know about him" where "him" is defined in a previous line) -- ignore items mentioned there that are not in the last line.

Return the output as a single comma-separated list, or NONE if there is nothing of note to return (e.g. the user is just issuing a greeting or having a simple conversation).

EXAMPLE
Conversation history:
Person #1: how's it going today?
AI: "It's going great! How about you?"
Person #1: good! busy working on Langchain. lots to do.
AI: "That sounds like a lot of work! What kind of things are you doing to make Langchain better?"
Last line:
Person #1: i'm trying to improve Langchain's interfaces, the UX, its integration

到此，关于memory done